Capítulo 6

6.3 Patrones de organización de datos.

6.3.2 Ordenación de registros

Prototipo 1 - Ordenación total Versión 1 - La ordenación se produce toda en el Reducer

In [ ]:
%%writefile ordenTotal.py
#!/usr/bin/env python

from mrjob.job import MRJob
import re
class ordenTotal(MRJob):
    
    #Inicializamos la estructura que nos permitirá contener los valores que nos llegan del Map
    def mapper_init(self):
        self.conjuntoRegistros = []
    
    def mapper(self,_, line):
        linea=line.rstrip('/n').split(';') 
    
        encontrado=re.search('[a-zA-Z]',linea[1])
                                                
        if encontrado==None:
            #Se agrega el registro completo a la lista
            self.conjuntoRegistros.append(linea) 
        
    def mapper_final(self):
        # mando el conjunto de registros al Reducer para reunirlos todos.
        yield None, self.conjuntoRegistros
        
        
 #===============================REDUCER=======================================     
        
    def reducer(self, _, values):
        listaCompletaRegistros=[]
        
        for valores in values:
            listaCompletaRegistros.extend(valores)
        #Expresión Lambda para indicar
        # por qué campo tiene que ordenar la lista.
        listaCompletaRegistros.sort(key = lambda precio: float(precio[5])) 
        
        for registro in listaCompletaRegistros: 
            yield _, registro
        
if __name__ == '__main__':
    ordenTotal.run()

In [ ]:
!python ordenTotal.py archivos_datos/Online_Retail.csv

Capítulo 6

6.3 Patrones de organización de datos.

6.3.2 Ordenación de registros

Prototipo 1 - Ordenación total Versión 2 - Igual que el anterior pero la ordenación se produce en cada uno de los Mapper, y después se hace en el Reducer

In [ ]:
%%writefile ordenTotalV2.py
#!/usr/bin/env python

from mrjob.job import MRJob
import sys, os, re

class ordenTotalV2(MRJob):
    
    #Inicializamos la estructura que nos permitirá contener los valores que nos llegan del Map
    def mapper_init(self):
        self.conjuntoRegistros = []
    
    def mapper(self,_, line):
        linea=line.rstrip('/n').split(';') 
        encontrado=re.search('[a-zA-Z]',linea[1])#Para que no me coja la línea de los nombres de campo. Cojo el primero mismo
                                                # si es una cadena de caracteres, no se agrega a la lista.
        #Preguntamos si ha encontrado la expresion regular (cadena de caracteres). Si no la ha encontrado, entonces
        #agregamos el registro al conjunto de registros que está generando el map.
        if encontrado==None:
            self.conjuntoRegistros.append(linea) #Aquí agrega el registro completo
        
    def mapper_final(self):
        # Si no desactivamos el reducer, se puede ver como salen los conjuntos de cada mapper distribuido en su ejecución
        # paralela en el cluster.
        # mando el conjunto de registros al Reducer para unificarlos en uno solo allí.
        
        #yield "REGISTROS PARA ORDENAR", self.conjuntoRegistros
        
        #Otra versión
        self.conjuntoRegistros.sort(key = lambda x: float(x[5]))
        for registro in self.conjuntoRegistros:
            yield None, registro
        
 #===============================REDUCER=======================================     
        
    def reducer(self, key, values):
        listaCompletaRegistros=[]
        listaCompletaRegistros.extend(values)
       
        listaCompletaRegistros.sort(key = lambda x: float(x[5])) #Usamos la expresión Lambda 
        
        for registro in listaCompletaRegistros:
            yield None, registro
        
if __name__ == '__main__':
    ordenTotalV2.run()

In [ ]:
!python ordenTotalV2.py archivos_datos/Online_Retail.csv

Capítulo 6

6.3 Patrones de organización de datos.

6.3.2 Ordenación de registros

Prototipo 2 - Ordenación agrupada con clave.Se ordenan los registros usando una clave, en este caso será el número de cliente. 

In [ ]:
%%writefile ordenTotalV3.py
#!/usr/bin/env python

from mrjob.job import MRJob
import re

class ordenTotalV3(MRJob):
    
    #Inicializamos la estructura que nos permitirá contener los valores que nos llegan del Map
    def mapper_init(self):
        self.conjuntoRegistros = []
    
    def mapper(self,_, line):
        self.linea=line.rstrip('/n').split(';') 
        
        #Para que no coja la línea de los nombres de campo.
        # si es una cadena de caracteres, no se agrega a la lista.
        encontrado=re.search('[a-zA-Z]',self.linea[1])
                                                
        if encontrado==None:
            #Se agrega el registro completo a la lista
            self.conjuntoRegistros.append(self.linea) 
        
    def mapper_final(self):
        # mando el conjunto de registros al Reducer para reunirlos todos.
        yield self.linea[6], self.conjuntoRegistros
        
        
 #===============================REDUCER=======================================     
        
    def reducer(self, cliente, values):
        listaCompletaRegistros=[]
        
        for valores in values:
            listaCompletaRegistros.extend(valores)
        
        # ordenamos la lista 
        listaCompletaRegistros.sort(key = lambda x: float(x[5])) 
        
        for registro in listaCompletaRegistros:
            yield cliente, registro
        
if __name__ == '__main__':
    ordenTotalV3.run()

In [ ]:
!python ordenTotalV3.py archivos_datos/Online_Retail.csv